# GCP BigQueryとGCP Functions、ChatGPTのGPTアクションを使用した構成

このノートブックでは、OpenAI埋め込みを使用してGoogle Cloud BigQueryをベクトル検索機能付きデータベースとして使用し、その上にGoogle Cloud Functionを作成してChatGPTのカスタムGPTに接続する手順を段階的に説明します。

これは、Google Cloud Platform（GCP）内に含まれるRAGインフラストラクチャを設定し、ChatGPTなどの他のプラットフォームと統合するためのエンドポイントとして公開したい顧客向けのソリューションとなります。

Google Cloud BigQueryは、Googleのインフラストラクチャの処理能力を使用して超高速SQLクエリを可能にする、完全管理型のサーバーレスデータウェアハウスです。開発者は大規模なデータセットを簡単に保存・分析できます。

Google Cloud Functionsは、サーバーやランタイム環境を管理することなく、クラウドイベントに応答する小さな単一目的の関数を作成できる、軽量でイベントベースの非同期コンピューティングソリューションです。

## 前提条件：

このクックブックを実行するには、以下が必要です：
- アクセス権限のあるGCPプロジェクト
- BigQueryデータセットとGoogle Cloud Functionを作成する権限を持つGCPユーザー
- [GCP CLI](https://cloud.google.com/sdk/docs/downloads-interactive)がインストールされ、接続されていること
- OpenAI APIキー
- ChatGPT Plus、Teams、またはEnterpriseサブスクリプション

## アーキテクチャ

以下は、このソリューションのアーキテクチャ図です。段階的に説明していきます：

![bigquery-rag-architecture.png](../../../../images/bigquery_rag_architecture.png)

## 目次

1. **[環境のセットアップ](#set-up-environment)** 必要なライブラリのインストールとインポート、GCP設定の構成により環境をセットアップします。含まれる内容：
    - [必要なライブラリのインストールとインポート](#install-and-import-required-libraries)
    - [GCPプロジェクトの設定](#configure-gcp-project)
    - [OpenAI設定の構成](#configure-openai-settings)

2. **[データの準備](#prepare-data)** ドキュメントを埋め込み、追加のメタデータを取得してアップロード用のデータを準備します。この例では、OpenAIのドキュメントのサブセットをサンプルデータとして使用します。

3. **[ベクトル検索付きBigQueryテーブルの作成](#create-bigquery-table-with-vector-search)**  
準備したデータをアップロードするBigQueryテーブルを作成します。含まれる内容：

    - [データセットの作成](#create-bigquery-dataset)：BigQueryでデータセットを作成する手順。
    - [テーブルの作成とデータのアップロード](#creating-table-and-upload-data)：BigQueryでテーブルを作成する手順。

4. **[GCP Functionの作成](#create-gcp-function)** gcloud CLIと事前に計算された環境変数を使用

5. **[ChatGPTのカスタムGPTへの入力](#input-in-a-custom-gpt-in-chatgpt)** BigQueryの埋め込みデータに対して検索を実行：

    - [ベクトル検索](#test-search)：ベクトルベースの検索クエリを実行する手順。
    - [メタデータフィルタリング検索](#perform-search-with-metadata-filtering)：メタデータフィルタリングを実行する手順。

# 環境のセットアップ

## 必要なライブラリのインストールとインポート
以下のライブラリは、標準Pythonライブラリ、サードパーティライブラリ、およびGCP関連ライブラリに分類できます。

In [ ]:
! pip install -q google-auth
! pip install -q openai
! pip install -q pandas
! pip install -q google-cloud-functions
! pip install -q python-dotenv
! pip install -q pyperclip
! pip install -q PyPDF2
! pip install -q tiktoken
! pip install -q google-cloud-bigquery
! pip install -q pyyaml

In [ ]:
# Standard Libraries
import json  
import os
import csv
import shutil
from itertools import islice
import concurrent.futures
import yaml

# Third-Party Libraries
import pandas as pd
import numpy as np
from PyPDF2 import PdfReader
import tiktoken
from dotenv import load_dotenv
import pyperclip

# OpenAI Libraries
from openai import OpenAI

# Google Cloud Identity and Credentials
from google.auth import default
from google.cloud import bigquery
from google.cloud import functions_v1


## GCPプロジェクトの設定

まだセットアップされていない場合は、GCP CLIをインストールし、GCPに認証を行い、デフォルトプロジェクトを設定します。

In [ ]:
# Add gcloud to PATH
os.environ['PATH'] += os.pathsep + os.path.expanduser('~/google-cloud-sdk/bin')

# Verify gcloud is in PATH
! gcloud --version

In [ ]:
! gcloud auth application-default login

In [ ]:
project_id = "<insert_project_id>"  # Replace with your actual project ID
! gcloud config set project {project_id}

In [ ]:
! gcloud services enable cloudfunctions.googleapis.com
! gcloud services enable cloudbuild.googleapis.com
! gcloud services enable bigquery.googleapis.com

## OpenAI設定の構成

このセクションでは、OpenAIの認証設定について説明します。このセクションを進める前に、OpenAI APIキーを取得していることを確認してください。

In [ ]:
openai_api_key = os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as an env var>") # Saving this as a variable to reference in function app in later step
openai_client = OpenAI(api_key=openai_api_key)
embeddings_model = "text-embedding-3-small" # We'll use this by default, but you can change to your text-embedding-3-large if desired

## ベクター検索機能を持つGCP BigQueryの設定

このセクションでは、BigQueryでデータセットを作成し、埋め込みとベクター検索に使用される浮動小数点のベクターを保存する方法について説明します。

In [ ]:
from google.auth import default

# Use default credentials
credentials, project_id = default()
region = "us-central1" # e.g: "us-central1"
print("Default Project ID:", project_id)

# データの準備
oai_docsフォルダ内のOpenAI docsの数ページを埋め込み、保存します。まず各ページを埋め込み、CSVに追加し、そのCSVを使用してインデックスにアップロードします。

[このクックブック](khttps://github.com/openai/openai-cookbook/blob/main/examples/Embedding_long_inputs.ipynb)で紹介されているいくつかの技術を使用します。これは、セクション、画像/グラフ/図表を説明するためのビジョンモデルの使用、長い文書のチャンク間でのテキストの重複などの変数を考慮せずに、テキストを埋め込む簡単な方法です。

8191トークンのコンテキストを超える長いテキストファイルを処理するために、チャンクの埋め込みを個別に使用するか、または平均化（各チャンクのサイズで重み付け）などの方法で組み合わせることができます。

Pythonの公式クックブックから、シーケンスをチャンクに分割する関数を使用します。

In [ ]:
def batched(iterable, n):
    """Batch data into tuples of length n. The last batch may be shorter."""
    # batched('ABCDEFG', 3) --> ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    it = iter(iterable)
    while (batch := tuple(islice(it, n))):
        yield batch


次に、文字列をトークンにエンコードし、それをチャンクに分割する関数を定義します。OpenAIによる高速なオープンソーストークナイザーであるtiktokenを使用します。

Tiktokenを使ったトークンの数え方について詳しく知りたい場合は、[このクックブック](https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken)をご確認ください。

In [ ]:
def chunked_tokens(text, chunk_length, encoding_name='cl100k_base'):
    # Get the encoding object for the specified encoding name. OpenAI's tiktoken library, which is used in this notebook, currently supports two encodings: 'bpe' and 'cl100k_base'. The 'bpe' encoding is used for GPT-3 and earlier models, while 'cl100k_base' is used for newer models like GPT-4.
    encoding = tiktoken.get_encoding(encoding_name)
    # Encode the input text into tokens
    tokens = encoding.encode(text)
    # Create an iterator that yields chunks of tokens of the specified length
    chunks_iterator = batched(tokens, chunk_length)
    # Yield each chunk from the iterator
    yield from chunks_iterator

最後に、入力テキストが最大コンテキスト長よりも長い場合でも、入力トークンをチャンクに分割し、各チャンクを個別に埋め込むことで、埋め込みリクエストを安全に処理する関数を書くことができます。`average`フラグを`True`に設定すると、チャンク埋め込みの重み付き平均を返し、`False`に設定すると、変更されていないチャンク埋め込みのリストを単純に返します。

> 注意：ここでは他の手法も使用できます：
> - GPT-4oを使用して画像/チャートの説明を埋め込み用にキャプチャする
> - 段落やセクションに基づいてチャンクに分割する
> - 各記事についてより詳細なメタデータを追加する

In [ ]:
EMBEDDING_CTX_LENGTH = 8191
EMBEDDING_ENCODING='cl100k_base'

In [ ]:
def generate_embeddings(text, model):
    # Generate embeddings for the provided text using the specified model
    embeddings_response = openai_client.embeddings.create(model=model, input=text)
    # Extract the embedding data from the response
    embedding = embeddings_response.data[0].embedding
    return embedding

def len_safe_get_embedding(text, model=embeddings_model, max_tokens=EMBEDDING_CTX_LENGTH, encoding_name=EMBEDDING_ENCODING):
    # Initialize lists to store embeddings and corresponding text chunks
    chunk_embeddings = []
    chunk_texts = []
    # Iterate over chunks of tokens from the input text
    for chunk in chunked_tokens(text, chunk_length=max_tokens, encoding_name=encoding_name):
        # Generate embeddings for each chunk and append to the list
        chunk_embeddings.append(generate_embeddings(chunk, model=model))
        # Decode the chunk back to text and append to the list
        chunk_texts.append(tiktoken.get_encoding(encoding_name).decode(chunk))
    # Return the list of chunk embeddings and the corresponding text chunks
    return chunk_embeddings, chunk_texts

次に、ドキュメントに関する追加のメタデータをキャプチャするヘルパー関数を定義できます。この例では、後でメタデータフィルターで使用するために、カテゴリのリストから選択します。

In [ ]:
categories = ['authentication','models','techniques','tools','setup','billing_limits','other']

def categorize_text(text, categories):

    # Create a prompt for categorization
    messages = [
        {"role": "system", "content": f"""You are an expert in LLMs, and you will be given text that corresponds to an article in OpenAI's documentation.
         Categorize the document into one of these categories: {', '.join(categories)}. Only respond with the category name and nothing else."""},
        {"role": "user", "content": text}
    ]
    try:
        # Call the OpenAI API to categorize the text
        response = openai_client.chat.completions.create(
            model="gpt-4o",
            messages=messages
        )

        # Extract the category from the response
        category = response.choices[0].message.content
        return category
    except Exception as e:
        print(f"Error categorizing text: {str(e)}")
        return None

# Example usage

次に、`oai_docs`フォルダ内の.txtファイルを処理するためのヘルパー関数を定義できます。ご自身のデータでも自由にお使いください。これは.txtファイルと.pdfファイルの両方をサポートしています。

In [ ]:
def extract_text_from_pdf(pdf_path):
    # Initialize the PDF reader
    reader = PdfReader(pdf_path)
    text = ""
    # Iterate through each page in the PDF and extract text
    for page in reader.pages:
        text += page.extract_text()
    return text

def process_file(file_path, idx, categories, embeddings_model):
    file_name = os.path.basename(file_path)
    print(f"Processing file {idx + 1}: {file_name}")
    
    # Read text content from .txt files
    if file_name.endswith('.txt'):
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
    # Extract text content from .pdf files
    elif file_name.endswith('.pdf'):
        text = extract_text_from_pdf(file_path)
    
    title = file_name
    # Generate embeddings for the title
    title_vectors, title_text = len_safe_get_embedding(title, embeddings_model)
    print(f"Generated title embeddings for {file_name}")
    
    # Generate embeddings for the content
    content_vectors, content_text = len_safe_get_embedding(text, embeddings_model)
    print(f"Generated content embeddings for {file_name}")
    
    category = categorize_text(' '.join(content_text), categories)
    print(f"Categorized {file_name} as {category}")
    
    # Prepare the data to be appended
    data = []
    for i, content_vector in enumerate(content_vectors):
        data.append({
            "id": f"{idx}_{i}",
            "vector_id": f"{idx}_{i}",
            "title": title_text[0],
            "text": content_text[i],
            "title_vector": json.dumps(title_vectors[0]),  # Assuming title is short and has only one chunk
            "content_vector": json.dumps(content_vector),
            "category": category
        })
        print(f"Appended data for chunk {i + 1}/{len(content_vectors)} of {file_name}")
    
    return data



これで、このヘルパー関数を使用してOpenAIドキュメントを処理します。以下の`process_files`でフォルダを変更することで、独自のデータを使用するように自由に更新してください。

なお、これは選択したフォルダ内のドキュメントを並行処理するため、txtファイルを使用する場合は30秒未満で完了し、PDFを使用する場合は少し長くかかります。

In [ ]:
## Customize the location below if you are using different data besides the OpenAI documentation. Note that if you are using a different dataset, you will need to update the categories list as well.
folder_name = "../../../data/oai_docs"

files = [os.path.join(folder_name, f) for f in os.listdir(folder_name) if f.endswith('.txt') or f.endswith('.pdf')]
data = []

# Process each file concurrently
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(process_file, file_path, idx, categories, embeddings_model): idx for idx, file_path in enumerate(files)}
    for future in concurrent.futures.as_completed(futures):
        try:
            result = future.result()
            data.extend(result)
        except Exception as e:
            print(f"Error processing file: {str(e)}")

# Write the data to a CSV file
csv_file = os.path.join("..", "embedded_data.csv")
with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ["id", "vector_id", "title", "text", "title_vector", "content_vector","category"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in data:
        writer.writerow(row)
        print(f"Wrote row with id {row['id']} to CSV")

# Convert the CSV file to a Dataframe
article_df = pd.read_csv("../embedded_data.csv")
# Read vectors from strings back into a list using json.loads
article_df["title_vector"] = article_df.title_vector.apply(json.loads)
article_df["content_vector"] = article_df.content_vector.apply(json.loads)
article_df["vector_id"] = article_df["vector_id"].apply(str)
article_df["category"] = article_df["category"].apply(str)
article_df.head()

これで、ベクターデータベースにアップロードできる6つの列を持つ`embedded_data.csv`ファイルが完成しました！

# Vector Search機能付きBigQueryテーブルの作成

## BigQueryデータセットの作成

Google SDKを活用して、「oai_docs」という名前のデータセットと「embedded_data」というテーブル名を作成しますが、これらの変数は自由に変更してください（リージョンも変更可能です）。

*注意: BigQueryインデックスは作成しません。これはベクトル検索のパフォーマンスを向上させる可能性がありますが、そのようなインデックスにはデータセット内に1,000行以上が必要であり、この例では該当しません。ただし、独自のユースケースでは自由に活用してください。*

In [ ]:
# Create bigquery table

from google.cloud import bigquery
from google.api_core.exceptions import Conflict

# Define the dataset ID (project_id.dataset_id)
raw_dataset_id = 'oai_docs'
dataset_id = project_id + '.' + raw_dataset_id

client = bigquery.Client(credentials=credentials, project=project_id)

# Construct a full Dataset object to send to the API
dataset = bigquery.Dataset(dataset_id)

# Specify the geographic location where the dataset should reside
dataset.location = "US"

# Send the dataset to the API for creation
try:
    dataset = client.create_dataset(dataset, timeout=30)
    print(f"Created dataset {client.project}.{dataset.dataset_id}")
except Conflict:
    print(f"dataset {dataset.dataset_id } already exists")
    

In [ ]:
# Read the CSV file, properly handling multiline fields
csv_file_path = "../embedded_data.csv"
df = pd.read_csv(csv_file_path, engine='python', quotechar='"', quoting=1)

# Display the first few rows of the dataframe
df.head()


## テーブルの作成とデータのアップロード

属性名と型を指定してテーブルを作成します。単一行に対してfloatのベクトルを格納できる'content_vector'属性に注目してください。これをベクトル検索で使用します。

このコードは、以前に作成したCSVをループして、Bigqueryに行を挿入します。
このコードを複数回実行すると、同一の行が複数回挿入され、検索時の精度が低下する可能性があります（IDに一意性制約を設けるか、毎回DBをクリーンアップすることをお勧めします）。

In [ ]:
# Read the CSV file, properly handling multiline fields
dataset_id = project_id + '.' + raw_dataset_id
client = bigquery.Client(credentials=credentials, project=project_id)
csv_file_path = "../embedded_data.csv"
df = pd.read_csv(csv_file_path, engine='python', quotechar='"', quoting=1)

# Preprocess the data to ensure content_vector is correctly formatted
# removing last and first character which are brackets [], comma splitting and converting to float
def preprocess_content_vector(row):
    row['content_vector'] = [float(x) for x in row['content_vector'][1:-1].split(',')]
    return row

# Apply preprocessing to the dataframe
df = df.apply(preprocess_content_vector, axis=1)

# Define the schema of the final table
final_schema = [
    bigquery.SchemaField("id", "STRING"),
    bigquery.SchemaField("vector_id", "STRING"),
    bigquery.SchemaField("title", "STRING"),
    bigquery.SchemaField("text", "STRING"),
    bigquery.SchemaField("title_vector", "STRING"),
    bigquery.SchemaField("content_vector", "FLOAT64", mode="REPEATED"),
    bigquery.SchemaField("category", "STRING"),
]

# Define the final table ID
raw_table_id = 'embedded_data'
final_table_id = f'{dataset_id}.' + raw_table_id

# Create the final table object
final_table = bigquery.Table(final_table_id, schema=final_schema)

# Send the table to the API for creation
final_table = client.create_table(final_table, exists_ok=True)  # API request
print(f"Created final table {project_id}.{final_table.dataset_id}.{final_table.table_id}")

# Convert DataFrame to list of dictionaries for BigQuery insertion
rows_to_insert = df.to_dict(orient='records')

# Upload data to the final table
errors = client.insert_rows_json(f"{final_table.dataset_id}.{final_table.table_id}", rows_to_insert)  # API request

if errors:
    print(f"Encountered errors while inserting rows: {errors}")
else:
    print(f"Successfully loaded data into {dataset_id}:{final_table_id}")

# 検索のテスト
データがアップロードされたので、以下でピュアなベクトル類似度検索とメタデータフィルタリングの両方をローカルでテストして、期待通りに動作することを確認します。

ピュアなベクトル検索とメタデータフィルタリングの両方をテストできます。

以下のクエリはピュアなベクトル検索で、カテゴリでフィルタリングを行いません。

In [ ]:
query = "What model should I use to embed?"
category = "models"

embedding_query = generate_embeddings(query, embeddings_model)
embedding_query_list = ', '.join(map(str, embedding_query))

query = f"""
WITH search_results AS (
  SELECT query.id AS query_id, base.id AS base_id, distance
  FROM VECTOR_SEARCH(
    TABLE oai_docs.embedded_data, 'content_vector',
    (SELECT ARRAY[{embedding_query_list}] AS content_vector, 'query_vector' AS id),
    top_k => 2, distance_type => 'COSINE', options => '{{"use_brute_force": true}}')
)
SELECT sr.query_id, sr.base_id, sr.distance, ed.text, ed.title
FROM search_results sr
JOIN oai_docs.embedded_data ed ON sr.base_id = ed.id
ORDER BY sr.distance ASC
"""

query_job = client.query(query)
results = query_job.result()  # Wait for the job to complete

for row in results:
    print(f"query_id: {row['query_id']}, base_id: {row['base_id']}, distance: {row['distance']}, text_truncated: {row['text'][0:100]}")


## メタデータフィルタリングを使用した検索の実行
メタデータフィルタリングにより、ベクトル検索の意味的に最も近い結果を取得することに加えて、特定の属性を持つ結果に絞り込むことができます。

提供されたコードスニペットは、メタデータフィルタリングを使用してクエリを実行する方法を示しています：

In [ ]:

query = "What model should I use to embed?"
category = "models"

embedding_query = generate_embeddings(query, embeddings_model)
embedding_query_list = ', '.join(map(str, embedding_query))


query = f"""
WITH search_results AS (
  SELECT query.id AS query_id, base.id AS base_id, distance
  FROM VECTOR_SEARCH(
    (SELECT * FROM oai_docs.embedded_data WHERE category = '{category}'), 
    'content_vector',
    (SELECT ARRAY[{embedding_query_list}] AS content_vector, 'query_vector' AS id),
    top_k => 4, distance_type => 'COSINE', options => '{{"use_brute_force": true}}')
)
SELECT sr.query_id, sr.base_id, sr.distance, ed.text, ed.title, ed.category
FROM search_results sr
JOIN oai_docs.embedded_data ed ON sr.base_id = ed.id
ORDER BY sr.distance ASC
"""


query_job = client.query(query)
results = query_job.result()  # Wait for the job to complete

for row in results:
    print(f"category: {row['category']}, title: {row['title']}, base_id: {row['base_id']}, distance: {row['distance']}, text_truncated: {row['text'][0:100]}")


# GCP関数の作成

## 変数のエクスポート

このフォルダ内の`main.py`にある関数をデプロイします（[こちら](https://github.com/openai/openai-cookbook/blob/main/examples/chatgpt/rag-quickstart/gcp/main.py)でも利用可能）。

最初のステップとして、テーブル/データセットをターゲットにし、OpenAIのAPIを使用してEmbeddingsを生成するための変数をエクスポートします。

In [ ]:
# Create a dictionary to store the environment variables (they were used previously and are just retrieved)
env_variables = {
    'OPENAI_API_KEY': openai_api_key,
    'EMBEDDINGS_MODEL': embeddings_model,
    'PROJECT_ID': project_id,
    'DATASET_ID': raw_dataset_id,
    'TABLE_ID': raw_table_id
}

# Write the environment variables to a YAML file
with open('env.yml', 'w') as yaml_file:
    yaml.dump(env_variables, yaml_file, default_flow_style=False)

print("env.yml file created successfully.")

## 関数のデプロイ

現在のプロジェクト用に「openai_docs_search」というGoogle関数を作成します。そのために、以下のCLIコマンドを実行し、先ほど作成した環境変数を活用します。この関数は認証なしでどこからでも呼び出すことができることに注意してください。本番環境では使用しないか、追加の認証メカニズムを追加してください。

In [ ]:
! gcloud functions deploy openai_docs_search \
  --runtime python39 \
  --trigger-http \
  --allow-unauthenticated \
  --env-vars-file env.yml

# ChatGPTのカスタムGPTでの入力

Vector Search Indexにクエリを実行するGCP Functionができたので、これをGPT Actionとして設定しましょう！

GPTについてのドキュメントは[こちら](https://openai.com/index/introducing-gpts/)、GPT Actionsについては[こちら](https://platform.openai.com/docs/actions)を参照してください。以下をGPTの指示として、またGPT ActionのOpenAPI仕様として使用してください。

## OpenAPI仕様の作成

以下はサンプルのOpenAPI仕様です。下記のブロックを実行すると、機能的な仕様がクリップボードにコピーされ、GPT Actionに貼り付けることができます。

なお、これはデフォルトでは認証機能がありませんが、GCPのドキュメント[こちら](https://cloud.google.com/functions/docs/securing/authenticating)に従ってGCP Functionsに認証を設定することができます。

In [ ]:
spec = f"""
openapi: 3.1.0
info:
  title: OpenAI API documentation search
  description: API to perform a semantic search over OpenAI APIs
  version: 1.0.0
servers:
  - url: https://{region}-{project_id}.cloudfunctions.net
    description: Main (production) server
paths:
  /openai_docs_search:
    post:
      operationId: openai_docs_search
      summary: Perform a search
      description: Returns search results for the given query parameters.
      requestBody:
        required: true
        content:
          application/json:
            schema:
              type: object
              properties:
                query:
                  type: string
                  description: The search query string
                top_k:
                  type: integer
                  description: Number of top results to return. Maximum is 3.
                category:
                  type: string
                  description: The category to filter on, on top of similarity search (used for metadata filtering). Possible values are {categories}.
      responses:
        '200':
          description: A JSON response with the search results
          content:
            application/json:
              schema:
                type: object
                properties:
                  items:
                    type: array
                    items:
                      type: object
                      properties:
                        text:
                          type: string
                          example: "Learn how to turn text into numbers, unlocking use cases like search..."
                        title:
                          type: string
                          example: "embeddings.txt"
                        distance:
                          type: number
                          format: float
                          example: 0.484939891778730
                        category:
                          type: string
                          example: "models"
"""
print(spec)
pyperclip.copy(spec)
print("OpenAPI spec copied to clipboard")


## GPT指示の作成

指示は必要に応じて自由に修正してください。プロンプトエンジニアリングのヒントについては、[こちら](https://platform.openai.com/docs/guides/prompt-engineering)のドキュメントをご確認ください。

In [ ]:
instructions = f'''
You are an OpenAI docs assistant. You have an action in your knowledge base where you can make a POST request to search for information. The POST request should always include: {{
    "query": "<user_query>",
    "k_": <integer>,
    "category": <string, but optional>
}}. Your goal is to assist users by performing searches using this POST request and providing them with relevant information based on the query.

You must only include knowledge you get from your action in your response.
The category must be from the following list: {categories}, which you should determine based on the user's query. If you cannot determine, then do not include the category in the POST request.
'''
pyperclip.copy(instructions)
print("GPT Instructions copied to clipboard")
print(instructions)

# まとめ

以下の手順により、GCP BigQuery Vector SearchとChatGPTのGPT Actionsの統合に成功しました：

1. OpenAIの埋め込みを使用してドキュメントを埋め込み、gpt-4oを使用して追加のメタデータを付与しました。
2. そのデータをGCP BigQuery（生データと埋め込みベクトル）にアップロードしました。
3. それらを取得するためのエンドポイントをGCP Functions上に作成しました。
4. カスタムGPTに組み込みました。

私たちのGPTは、ユーザーのクエリに答えるための情報を取得できるようになり、データに対してより正確でカスタマイズされた回答が可能になりました。以下がGPTの動作例です：

![gcp-rag-quickstart-gpt.png](../../../../images/gcp_rag_quickstart_gpt.png)